# Tests

## Imports and stuff

In [1]:
import sys

def append_path(s):
    if s in sys.path:
        return
    sys.path.append(s)

append_path("..")
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from numpy.random import default_rng
rng = default_rng()
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import networkx as nx
import torch
from torch import Tensor


In [3]:
import ml_lib


## Datasets

In [4]:
from ml_lib.datasets import register as dataset_register, transform_register

In [5]:
dataset0 = dataset_register["Torus4D"](1000)

In [6]:
dataset1 = transform_register["CacheTransform"](dataset0)

In [7]:
#dataset0[0]

In [8]:
dataset1[0]

tensor([[ 0.1680, -0.9947, -0.8321,  0.5713]])

In [9]:
dataset2 = transform_register["RenameTransform"](dataset1, {"_": "x"})
dataset2[0]

datapoint(x=tensor([[ 0.1680, -0.9947, -0.8321,  0.5713]]))

In [10]:
dataset3 = transform_register["RenameTransform"](dataset2, {"x": "x", 0:"y"})
dataset3[0]

datapoint(x=tensor([[ 0.1680, -0.9947, -0.8321,  0.5713]]), y=tensor([[ 0.1680, -0.9947, -0.8321,  0.5713]]))

In [11]:
normalize = lambda x : x / x.norm()
dataset4 = transform_register["MultipleFunctionTransform"](dataset3, {"x": normalize})
dataset4[0]

datapoint(x=tensor([[ 0.1177, -0.6970, -0.5831,  0.4003]]), y=tensor([[ 0.1680, -0.9947, -0.8321,  0.5713]]))

great!

## Environments

In [12]:
from ml_lib.environment import Environment, Scope, scopevar_of_str, str_of_scopevar, HierarchicEnvironment, ScopedEnvironment

In [13]:
scopevar_of_str("a/b/c")

(('a', 'b'), 'c')

In [14]:
str_of_scopevar(('a', 'b'), 'c')

'a/b/c'

In [15]:
env = Environment()

In [16]:
env

Environment(defaultdict(<class 'dict'>, {'_ipython_canary_method_should_not_exist_': {}, '_ipython_display_': {}, '_repr_mimebundle_': {}}))

In [17]:
env.record("hello", 1)

In [18]:
env.data

defaultdict(dict,
            {'_ipython_canary_method_should_not_exist_': {},
             '_ipython_display_': {},
             '_repr_mimebundle_': {},
             '_repr_html_': {},
             '_repr_markdown_': {},
             '_repr_svg_': {},
             '_repr_png_': {},
             '_repr_pdf_': {},
             '_repr_jpeg_': {},
             '_repr_latex_': {},
             '_repr_json_': {},
             '_repr_javascript_': {},
             'hello': {(): 1}})

In [19]:
env.record("world", 4, ("some", "scope"))
env.data

defaultdict(dict,
            {'_ipython_canary_method_should_not_exist_': {},
             '_ipython_display_': {},
             '_repr_mimebundle_': {},
             '_repr_html_': {},
             '_repr_markdown_': {},
             '_repr_svg_': {},
             '_repr_png_': {},
             '_repr_pdf_': {},
             '_repr_jpeg_': {},
             '_repr_latex_': {},
             '_repr_json_': {},
             '_repr_javascript_': {},
             'hello': {(): 1},
             'world': {('some', 'scope'): 4}})

In [20]:
env.get("hello")

1

In [21]:
env.get("world")

4

In [22]:
#import pdb; pdb.set_trace()
env.get("world", scope=("some",))

4

In [23]:
env.get("world", scope=("soe",))

In [24]:
hier_env = HierarchicEnvironment(parent=env)

In [25]:
hier_env.record("world", 2, ("some",))
hier_env.get("world", scope=("some",))


2

In [26]:
hier_env.get("world", scope=("some","scope"))


4

In [27]:
def f(world, hello=6, magic=3):
    print(world, hello, magic)
    return 1

In [28]:
hier_env.run_function(f)

2 1 3


1

## Training

In [32]:
from ml_lib.models import Model
from ml_lib.pipeline import Trainer

In [33]:
class TestModel(Model):
    def predict(self, x):
        print(f"called predict with argument {x}")
        
    def compute_loss(self, x, y):
        print(f"compute_loss called with {x=}, {y=}")
        
model = TestModel() 

In [36]:
trainer = Trainer(model, dataset4, n_epochs = 4)

NameError: name 'loader' is not defined